In [1]:
!pip install -U keras-hub
!pip install -U keras
!pip install rouge-score

ERROR: Could not find a version that satisfies the requirement rouge-score (from versions: none)
ERROR: No matching distribution found for rouge-score


# Download llm model

In [2]:
import keras
import keras_hub
import numpy as np


2025-04-27 13:14:02.684551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745759642.910098      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745759642.974078      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
gemma_lm = keras_hub.models.GemmaCausalLM.from_preset("gemma_2b_en")

2025-04-27 13:14:20.242279: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [ ]:
prompt_template = """
<start_of_turn>user
You are an intelligent AI specialized in generating SQL queries.
Your task is to assist users in formulating SQL queries to retrieve specific information from a database.
Please provide the SQL query corresponding to the given prompt and context:

Prompt:
Convert a PostgreSQL query into ClickHouse dialect using PG query

Context:
PG_QUERY: Select * from metric_store.metrics<end_of_turn>
<start_of_turn>model
"""
generated = gemma_lm.generate(prompt_template, max_length=100)

In [ ]:
generated

Модель не знает синтаксиса clickhouse, ее будет сбивать промпт, указанный выше. Упростим его:

In [ ]:
prompt_template = """
<start_of_turn>user
Please provide the SQL query corresponding to the given prompt and context:

Prompt:
Convert a SQL query into other dialect using PG query

Context:
PG_QUERY: Select * from metric_store.metrics<end_of_turn>
<start_of_turn>model
"""
generated = gemma_lm.generate(prompt_template, max_length=100)

In [ ]:
generated

Некорректно считать какие-то метрики, зная что модель не умеет решать поставленную задачу. Но нужно замерить score базовой модели, чтобы сравнивать ее с предобученной

# Prepare dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='/kaggle/input/sql-to-clickhouse/test_dataset.csv')
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['natural_languge', 'pg_db_schema', 'pg_query', 'ch_db_schema', 'ch_query'],
        num_rows: 531
    })
})

In [5]:
prompt_template = """
<start_of_turn>user
Please provide the SQL query corresponding to the given prompt and context:

Prompt:
Convert a SQL query into other dialect using PG query

Context:
{sql_context} <end_of_turn>
<start_of_turn>model
"""

def generate_prompt(data_point):
    """Generate input text based on a prompt, task instruction, (context info), and answer.

    :param data_point: dict: Data point
    :return: dict: Data point with the added "prompt" field
    """
    sql_context = """
    Natural languge of query: {natural_languge},
    PostgreSQL query: {pg_query},
    """.format(
        natural_languge=data_point["natural_languge"],
        pg_query=data_point["pg_query"],
    )
    prompt_text = prompt_template.format(sql_context=sql_context)
    data_point["prompt"] = prompt_text

    return data_point

dataset = dataset.map(generate_prompt)

Map:   0%|          | 0/531 [00:00<?, ? examples/s]

In [6]:
test_dataset = dataset['train']

# Calculate model scores

In [7]:
import re
from collections import Counter
import numpy as np

def get_ngrams(text, n):
    """Извлекает n-граммы из текста (слова разделяются по пробелам)"""
    words = re.findall(r'\w+', text.lower())
    return [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]

def rouge_n(reference, candidate, n=1):
    """
    Вычисляет ROUGE-N (Precision, Recall, F1) между эталоном и кандидатом.
    """
    ref_ngrams = get_ngrams(reference, n)
    cand_ngrams = get_ngrams(candidate, n)
    
    ref_counter = Counter(ref_ngrams)
    cand_counter = Counter(cand_ngrams)
    
    # Общие n-граммы
    overlap = sum((ref_counter & cand_counter).values())
    
    precision = overlap / max(len(cand_ngrams), 1e-10)
    recall = overlap / max(len(ref_ngrams), 1e-10)
    f1 = 2 * precision * recall / max(precision + recall, 1e-10)
    
    return {'precision': precision, 'recall': recall, 'f1': f1}

def rouge_l(reference, candidate):
    """
    Упрощённый ROUGE-L (на основе самой длинной общей подпоследовательности слов)
    """
    ref_words = re.findall(r'\w+', reference.lower())
    cand_words = re.findall(r'\w+', candidate.lower())
    
    # Длина наибольшей общей подпоследовательности (LCS)
    lcs_length = len(_longest_common_subsequence(ref_words, cand_words))
    
    precision = lcs_length / max(len(cand_words), 1e-10)
    recall = lcs_length / max(len(ref_words), 1e-10)
    f1 = 2 * precision * recall / max(precision + recall, 1e-10)
    
    return {'precision': precision, 'recall': recall, 'f1': f1}

def _longest_common_subsequence(a, b):
    """Алгоритм LCS для двух списков"""
    lengths = [[0] * (len(b)+1) for _ in range(len(a)+1)]
    for i, x in enumerate(a):
        for j, y in enumerate(b):
            if x == y:
                lengths[i+1][j+1] = lengths[i][j] + 1
            else:
                lengths[i+1][j+1] = max(lengths[i+1][j], lengths[i][j+1])
    
    result = []
    i, j = len(a), len(b)
    while i > 0 and j > 0:
        if lengths[i][j] == lengths[i-1][j]:
            i -= 1
        elif lengths[i][j] == lengths[i][j-1]:
            j -= 1
        else:
            result.append(a[i-1])
            i -= 1
            j -= 1
    return result[::-1]


def evaluate_rouge_manual(dataset, model):
    results = {'rouge-1': [], 'rouge-2': [], 'rouge-l': []}
    
    for item in dataset:
        generated = model.generate(item["prompt"], max_length=len(item["prompt"]) + 100)
        model_answer = generated.split("model")[1].strip()
        reference = item["ch_query"]
        
        r1 = rouge_n(reference, generated, 1)
        r2 = rouge_n(reference, generated, 2)
        rl = rouge_l(reference, generated)
        
        results['rouge-1'].append(r1['f1'])
        results['rouge-2'].append(r2['f1'])
        results['rouge-l'].append(rl['f1'])

    avg_results = {
        metric: np.mean(scores) 
        for metric, scores in results.items()
    }
    return avg_results

In [9]:
# Применение к вашей модели
rouge_scores = evaluate_rouge_manual(test_dataset.select(range(40)), gemma_lm)
print("Средние ROUGE-F1:")
for metric, score in rouge_scores.items():
    print(f"{metric}: {score:.4f}")

Средние ROUGE-F1:
rouge-1: 0.1213
rouge-2: 0.0772
rouge-l: 0.1157
